# 02) 비디오 데이터 다루기

In [ ]:
#r "nuget: OpenCvSharp4"
#r "nuget: OpenCvSharp4.runtime.win"

using OpenCvSharp;

### # 비디오 출력
- OpenCV를 활용하면 비디오 파일을 쉽게 읽고 처리하는 것이 가능
    - 영상 파일, 웹캠, IP 카메라(실시간 스트리밍), 이미지 시퀀스 등
    - IP 스트리밍
        - protocol://host:port/script_name?script_params|auth와 같은 형태의 `URL`
    - 이미지 시퀀스
        - img_%02d.jpg와 같은 `문자열 포맷팅`
        - img_{두 자리 숫자}로 구성된 이미지 파일을 읽음
            - ex) img_00.jpg, img_01.jpg, img_02.jpg ...
- 비디오 파일은 연속적인 이미지 프레임으로 구성되며, 이를 순차적으로 읽어와서 처리
- `FFMPEG` 라이브러리를 지원하므로 다양한 형식의 비디오 파일(AVI, MP4 등) 핸들링 가능

- ```csharp
    VideoCaputre(string fileName)
  ```
    - `filename`: 파일 경로 + 경로
    - 비디오 로드 성공 여부를 반환하지 않음
        - 파일 경로가 올바르지 않거나 해당 파일의 코덱을 올바르게 인식하지 못해도 예외나 에러를 발생시키지 않음
        - 비디오 파일이 정상적으로 열렸는지 확인 필요
    - `VideoCapture` 객체는 비디오 파일의 정보를 저장하고, 현재 비디오 파일과 관련된 설정 및 상태를 관리하는 사용
    - 프레임은 이미지와 유사하게 표현되며, 일반적으로 Mat 클래스를 사용해 저장

In [ ]:
VideoCapture capture = new VideoCapture("../../media/Cock.mp4");

capture

- __프레임을 저장할 Mat 형식의 frame이 필요__
- 비디오는 `프레임(이미지)의 연속`이며, 모든 프레임을 순차적으로 표시하기 위해 반복문 사용
    - 비디오 파일은 끝이 존재하는 유한한 프레임을 갖음
    - 표시할 프레임이 모두 소진됐는데 계속해서 출력을 시도하면 오류가 발생
    - 이를 방지하기 위해 `PosFrames(현재 프레임의 수)`와 `FrameCount(총 프레임 수)`를 비교.
        - 현재 프레임이 끝에 도달하면, 비디오 파일을 다시 로드하거나 반복문을 종료

- ```csharp
    VideoCapture.Read(Mat frame)
  ```
    - 비디오 파일에서 프레임을 가져와 압축을 해제한 후, 해당 이미지를 Mat 클래스에 저장
        - 처음부터 모든 프레임을 가지고 있는 것이 아니라 그때그때 필요한 프레임을 가져옴
    - 이미지로 저장된 프레임은 `Cv2.Imshow`를 사용해 윈도에 표시
    - **비디오는 프레임(이미지)의 연속이므로 시간이 흐름에 따라 갱신**
    - 비디오 종료를 위해서 `Cv2.WaitKey`를 활용하여 일정 시간(33ms)만큼 대기

- ```csharp
    VideoCapture.Release()
  ```
    - 비디오 재생이 끝났으면, 리소스 관리를 위해 비디오 파일을 닫음
    - `OpenCV에서 사용한 자원을 해제`하고 `메모리 누수를 방지`

In [ ]:
Mat frame = new Mat();

while (true) {
    if(capture.PosFrames == capture.FrameCount) capture.Open("../../media/Cock.mp4");

    capture.Read(frame);

    Cv2.ImShow("VideoFrame", frame);

    if(Cv2.WaitKey(33) == 'q') break;
}

capture.Release();
Cv2.DestroyAllWindows();

### # 프레임 속도

- `프레임 속도(Frame Per Second, FPS)`는 매끄러운 영상 재생을 걸정짓는 중요한 요소 중 하나
    - "초당 몇 장의 이미지가 출력되는가?"를 의미
- 높은 값을 갖을수록 더 부드럽고 자연스러운 영상 출력이 가능
- 낮은 값일수록 영상이 끊기고 부자연스러움
- 영상 처리에서 FPS를 설정하거나 조절하기 위해 Cv2.WaitKey를 활용
    - 프레임 간의 대기 시간을 조절하여 프레임 간격을 제어
- FPS 계산 방식
    - $fps = \frac{1000}{interval}$
    - interval의 단위는 ms

### # 카메라 출력

- 위에서 연습한 `비디오 출력` 방법과 매우 유사
- 영상 파일 대신 **카메라의 장치 번호**를 사용
- 사용중인 플랫폼에서 카메라에 대한 접근 권한이 허용된 상태에서 가능

- ```csharp
    VideoCapture(int index);
  ```
    - `index`: 카메라 장치 번호
      - 일반적으로 0부터 시작해 순차적으로 증가
    - 내장 웹캠의 경우, 보통 0번을 사용

In [ ]:
VideoCapture capture = new VideoCapture(0);

capture

In [ ]:
Mat frame = new Mat();

capture.Set(VideoCaptureProperties.FrameWidth, 1280);
capture.Set(VideoCaptureProperties.FrameHeight, 960);

In [ ]:
while(true) {
    if(capture.IsOpened() == true) {
        capture.Read(frame);
        Cv2.ImShow("VideoFrame", frame);

        if(Cv2.WaitKey(33) == 'q') break;
    }
}

capture.Release();
Cv2.DestroyAllWindows();

- 영상 파일 출력과 카메라를 이용한 실시간 출력의 차이
    - `프레임 크기의 제한`
        - 카메라에 종속적
        - 설정한 프레임 크기가 카메라가 지원하는 해상도에 맞지 않는 다면, OpenCV는 가장 근사한 해상도로 조정
        - ex) [640 x 110]과 같이 비율이 맞지 않는 크기를 설정하면 110과 비율이 근사한 144로 변경되고, 114와 프레임 비율이 맞도로 640이 176으로 변경됨 -> [176 x 144]로 최종 변경
        - 프레임의 너비나 높이 중 더 낮은 값을 기준으로 비율에 맞는 크기로 변경경
    - `멀티 헤드 카메라 지원`
        - 멀티 헤드 카메라(Multi-head camera) 지원
            - 파노라마 촬용 등을 위해 제작된 특수 카메라
        - 프레임을 가져오는 방법에 차이가 있음
            - VideoCapture.Open
                - 카메라 장치를 읽음
            - `VideoCapture.Grab`
                - 비디오 파일에서 프레임을 읽어 내부 버퍼에 저장
                - 하드웨어 동기화가 이루어지지 않는 카메라 또는 멀티 헤드 카메라와 같이 여러 개의 헤드가 있는 카메라를 사용할 때 유용
            - `VideoCapture.Retrive`
                - 멀티 헤드 카메라의 프레임(그랩 메서드로 그랩한 데이터)을 디코딩한 후 반환
                - 플래그를 사용해 어떤 프레임을 가져올지 지정하는 것이 가능
            - 그랩과 리트리브 단계를 분리시킴으로써 `데이터의 복사 및 디코딩을 분리해 성능을 최적화`하는 것이 가능
                - 메모리 복사 최적화

### # 참고
[VideoCapture](https://docs.opencv.org/4.x/d8/dfe/classcv_1_1VideoCapture.html#a473055e77dd7faa4d26d686226b292c1)